# SeqApiPop analyses: RFMix

- Phasing with SNPs
    - Select SNPs
    - VCFs per Chromosme
    - Phasing with Shapeit
        - Phasing
        - Convert phased genotype back to vcf
        - Concatenate VCFs
        
- RFMix
- Select reference and query Samples
    - Make vcf files
- Make genetic maps
    - Find crossing-over in data from Liu et al., 2015
- Run RFMix
    - Couldn't get chromosome 1 to work
    
    
## Phasing with shapeit

### Select SNPs

- MAF > 1
- Max alleles = 2
- Chromosomes as numbers
    








Firstly, the names of the chromosomes must be changed to numbers from 1 to 16 because we do not include the mitochondrial chromosome

here is the script "ChangeChr" : to change the chromosome names into numbers:

```bash
#!/bin/sh


module load bioinfo/bcftools-1.9



VCF=MetaGenotypesCalled403_raw_snps_filtre_isec_plink.vcf.gz


echo "NC_037638.1 1" >> chr_name_conv.txt
echo "NC_037639.1 2" >> chr_name_conv.txt
echo "NC_037640.1 3" >> chr_name_conv.txt
echo "NC_037641.1 4" >> chr_name_conv.txt
echo "NC_037642.1 5" >> chr_name_conv.txt
echo "NC_037643.1 6" >> chr_name_conv.txt
echo "NC_037644.1 7" >> chr_name_conv.txt
echo "NC_037645.1 8" >> chr_name_conv.txt
echo "NC_037646.1 9" >> chr_name_conv.txt
echo "NC_037647.1 10" >> chr_name_conv.txt
echo "NC_037648.1 11" >> chr_name_conv.txt
echo "NC_037649.1 12" >> chr_name_conv.txt
echo "NC_037650.1 13" >> chr_name_conv.txt
echo "NC_037651.1 14" >> chr_name_conv.txt
echo "NC_037652.1 15" >> chr_name_conv.txt
echo "NC_037653.1 16" >> chr_name_conv.txt
echo "NC_001566.1 17" >> chr_name_conv.txt


bcftools annotate --rename-chrs chr_name_conv.txt $VCF | bgzip > rename.vcf.gz

bcftools index rename.vcf.gz
```

Here, the script for the selection of SNPs:

```bash
#!/bin/bash

#makeVCF.bash

#select from the vcf with chromosomes indicated as numbers, for plinkAnalyses
#filter on MAF > 0.01
#nb alleles max = 2

module load -f /home/gencel/vignal/save/000_ProgramModules/program_module

#Select the 403 samples and filter on MAF001

bcftools view  --samples-file /home/agirardon/work/seqapipopOnHAV3_1/RFmix/IndsAll403.list \
            --min-af 0.01:minor \
            --max-alleles 2 \
            --output-file /home/agirardon/work/seqapipopOnHAV3_1/RFmix/out/SeqApipop_403_MAF001_diAllelic_plink.vcf.gz \
            --output-type z \
            /home/agirardon/work/seqapipopOnHAV3_1/RFmix/rename.vcf.gz
bcftools index /home/agirardon/work/seqapipopOnHAV3_1/RFmix/out/SeqApipop_403_MAF001_diAllelic_plink.vcf.gz 

```

### VCFs per chromosome

- Shapeit requires one vcf per chromosome

Indeed, Shapeit needs an ascending order for SNPs positions, and in the VCF with all chromosomes the positions are in ascending order for one chromosome, but when moving to the next chromosome the position returns to the beginning of that chromosome 

```bash
#!/bin/bash

#separateChromosomes.sh

module load -f /home/gencel/vignal/save/000_ProgramModules/program_module


#Select the 403 samples and filter on MAF001

for i in  $(seq 1 16)

do

bcftools view  --regions ${i} \
        --output-file /home/agirardon/work/seqapipopOnHAV3_1/RFmix/out/SeqApipop_403_MAF001_diAllelic_plink_chr${i}.vcf.gz \
       /home/agirardon/work/seqapipopOnHAV3_1/RFmix/out/SeqApipop_403_MAF001_diAllelic_plink.vcf.gz
bcftools index -f /home/agirardon/work/seqapipopOnHAV3_1/RFmix/out/SeqApipop_403_MAF001_diAllelic_plink_chr${i}.vcf.gz
done

for i in  $(seq 1 16)
do
bcftools index -f /home/agirardon/work/seqapipopOnHAV3_1/RFmix/out/SeqApipop_403_MAF001_diAllelic_plink_chr${i}.vcf.gz
done
```

### Phasing with Shapeit

#### Phasing

Remark: the vcf file will be renamed, because Shapeit unzip the files, if they have .gz in their name. that's why .gz is only removed from the VCF name.

```bash
#!/bin/bash

#phasingShapeit.bash

module load bioinfo/shapeit.v2.904



for i in  $(seq 1 16)

do

sbatch --mem=50g --wrap="shapeit --input-vcf SeqApipop_403_MAF001_diAllelic_plink_chr${i}.vcf --force -O SeqApiPop_403_MAF001_diAllelic_phased_chr${i}.vcf"

done
```

#### Convert phased genotypes back to vcf

Each loop was launched one after the other 

``` bash
#!/bin/bash

#convertToVcf.bash

module load bioinfo/shapeit.v2.904
module load bioinfo/bcftools-1.9

for i in  $(seq 1 16)

do



sbatch --wrap="shapeit -convert --input-haps SeqApiPop_403_MAF001_diAllelic_phased_chr${i}.vcf --output-vcf SeqApiPop_403_MAF001_diAllelic_phased_chr${i}.vcf"

done

for i in  $(seq 1 16)

do

sbatch --wrap="bgzip SeqApiPop_403_MAF001_diAllelic_phased_chr${i}.vcf"
```

#### Concatenate VCFs 


- Make list

```bash
#!/bin/sh 

ls ~/work/seqapipopOnHAV3_1/RFmix/out/*phased*.vcf.gz > vcf.list

```

File vcf.list look like : 

```
/home/agirardon/work/seqapipopOnHAV3_1/RFmix/out/SeqApiPop_403_MAF001_diAllelic_phased_chr10.vcf.gz
/home/agirardon/work/seqapipopOnHAV3_1/RFmix/out/SeqApiPop_403_MAF001_diAllelic_phased_chr11.vcf.gz
/home/agirardon/work/seqapipopOnHAV3_1/RFmix/out/SeqApiPop_403_MAF001_diAllelic_phased_chr12.vcf.gz

```

The chromosomes do not come out in the right order, just cut and paste to reorder

## RFMix 

### Select reference and query Samples

Select from an Admixture Q matrix with K = 3 the individuals with > 0.93 pure backgrounds as reference => IndsReference.list The other samples => IndsQuery.list

#### Make BCF files

- References


```bash
#!/bin/bash

#selectBcfRef.bash

module load bioinfo/bcftools-1.9

#Select the 403 samples and filter on MAF001

bcftools view  --samples-file ~/work/seqapipopOnHAV3_1/RFmix/IndsAll403ref.list  \
        --output-file ~/work/seqapipopOnHAV3_1/RFmix/out/SeqApiPop_320_MAF001_diAllelic_phased_Ref.bcf \
        --output-type b \
        ~/work/seqapipopOnHAV3_1/RFmix/out/SeqApiPop_403_MAF001_diAllelic_phased_All.vcf.gz
bcftools index SeqApiPop_320_MAF001_diAllelic_phased_Ref.bcf
```

- Query

```bash
#!/bin/bash

#selectBcfQuery.bash

module load bioinfo/bcftools-1.9

bcftools view  --samples-file ~/work/seqapipopOnHAV3_1/RFmix/IndsAll403_query.list \
            --output-file ~/work/seqapipopOnHAV3_1/RFmix/out/SeqApiPop_403_MAF001_diAllelic_phased_Query.bcf \
            --output-type b \
            ~/work/seqapipopOnHAV3_1/RFmix/out/SeqApiPop_629_MAF001_diAllelic_phased_All.vcf.gz
bcftools index SeqApiPop_403_MAF001_diAllelic_phased_Query.bcf
```



## Make a genetic maps

To make the genetic map, here we simply recovered the same genetic map that Alain Vignal used for his RFmix analysis 

Find crossing-overs in data from Liu et al., 2015

Liu H, Zhang X, Huang J, Chen J-Q, Tian D, Hurst LD, et al. Causes and consequences of crossing-over evidenced via a high-resolution recombinational landscape of the honey bee. Genome Biol. 2015 Jan 1;16:15.

Reads from the project SRP043350 (Liu et al., 2015) were retrieved from Short Read Archive (SRA) (https://www.ncbi.nlm.nih.gov/sra), aligned to the reference genome for SNP detection (GATK pipeline): vcf file LiuGenotypesSNPs2allelesForCoSearchDuplOK.vcf.gz

The script find_crossing_overs.py was then used to detect crossing overs:

    -l Colony*.list : list of samples in the Colony
    -q SRR* : the colony's queen
    -c 6 : number of crossing-overs allowed simultaneously over all individuals

Then a genetic map was constructed from the data:

After importing the 3 files Colony3.list, make table with one line when multiple COs at one position and add column indicating the lines that are involved in double COs.

   - A double CO is due either to:
        - A non-crossing-over event marked by a single SNP
        - non-crossing-over events marked by more than one SNP will be detected by the distance to the previous and/or the next CO in the same individual. The distance is set here at 10 kb, as suggested by Liu, et al., 2015.
        -  A genotyping error (especially in the case of nb_COs = 1 or 2)
        -  nb_COs = number of ofspring from a colony, with COs at the same position
        - Problems in the assembly, such as small inversions, especially in the case of nb_COs > 2)
   - Detection of double crossing-overs: two lines must be removed
       - Line for the recombinant before the SNP : the previous and the next vectors are identical
       - Line for the recombinant after the SNP : the current examined vector and the vectors two steps back are identical



## Run RFmix

Add population colums to IndsAll403ref.list -> IndsAll403popref.list



```
head IndsAll403popref

Ab-PacBio	Black
BER10	Yellow
BER11	Yellow
BER12	Yellow
BER13	Yellow
BER14	Yellow
BER15	Yellow
BER16	Yellow
BER18	Yellow
BER19	Yellow
```

So we create a script to run the RFmix script : LanceRunRFmix.sh
    
```bash
#!/bin/bash

#LanceRunRFMix.bash

for i in  $(seq 1 16)
do
sbatch --mem=30g ~/work/seqapipopOnHAV3_1/RFmix/runRFMixWithGenetMap.bash ${i}

done
```


This allows us to run the following script, launching the RFmix on all our chromosomes : runRFMixWithGenetMap.bash



```bash

#!/bin/bash

module load bioinfo/rfmix-9505bfa
module load bioinfo/bcftools-1.9

rfmix -f ~/work/seqapipopOnHAV3_1/RFmix/out/SeqApiPop_403_MAF001_diAllelic_phased_Query.bcf \
        -r ~/work/seqapipopOnHAV3_1/RFmix/out/SeqApiPop_320_MAF001_diAllelic_phased_Ref.bcf \
        --chromosome=${1} \
        -m ~/work/seqapipopOnHAV3_1/RFmix/IndsAll403popref.list \
        -g ~/work/seqapipopOnHAV3_1/RFmix/GenetMap_march_2021_AV.txt \
        -o ~/work/seqapipopOnHAV3_1/RFmix/out2/SeqApiPopRFmixChr${1}
        --reanalyze-reference 
    
```


Example output :
    
- SeqApiPopRFmixChr10.fb.tsv
- SeqApiPopRFmixChr10.msp.tsv
- SeqApiPopRFmixChr10.rfmix.Q
- SeqApiPopRFmixChr10.sis.tsv